<h1>Análisis de Sentimiento

<h2>Importaciones

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import pyarrow

<h2>User Reviews Exploded

Se carga como dataframe el dataset previamente trabajado.

In [5]:
userReviewsExplodedDf = pd.read_parquet('../data/userReviewsExplodedPreReview.parquet')

In [3]:
userReviewsExplodedDf.head()

,user_id,user_url,reviews,review,recommend,item_id,review_date,review_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",Simple yet with great replayability. In my opi...,True,1250,2011-11-05,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",It's unique and worth a playthrough.,True,22200,2011-07-15,2011
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",Great atmosphere. The gunplay can be a bit chu...,True,43110,2011-04-21,2011
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'helpful': '15 of 20 people (75%...",I know what you think when you see this title ...,True,251610,2014-06-24,2014
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'helpful': '0 of 1 people (0%) f...",For a simple (it's actually not all that simpl...,True,227300,2013-09-08,2013


Se crea una función para el análisis de sentimiento. Para efectos de este análisis se utiliza VADER (Valence Aware Dictionary and sEntiment Reasoner) que es conocido por su eficacia en la detección de sentimientos en redes sociales y opiniones en línea. Se asignan valores de 0 a reseñas negativas, 1 a reseñas neutras o sin texto, y 2 a reseñas positivas.

In [7]:
def analyzeSentiment(text):
    if text is None:
        return '1'

    sia = SentimentIntensityAnalyzer()
    sentimentScore = sia.polarity_scores(text)['compound']

    if sentimentScore >= 0.05:
        return '2'
    elif sentimentScore <= -0.05:
        return '0'
    else:
        return '1'

Aplicamos el análisis de sentimiento a la columna de 'review' y se almacena el resultado en una columna nueva.

In [8]:
userReviewsExplodedDf['sentiment_analysis'] = userReviewsExplodedDf['review'].apply(analyzeSentiment)

Eliminamos las columnas que no nos servirán para las funciones y transformamos a parquet.

In [9]:
del userReviewsExplodedDf['reviews']
del userReviewsExplodedDf['review']
del userReviewsExplodedDf['user_url']

In [10]:
userReviewsExplodedDf.sort_values(by='sentiment_analysis').head()

,user_id,recommend,item_id,review_date,review_year,sentiment_analysis
40697,76561198068130802,False,304930,2015-07-05,2015,0
54665,76561198081924175,False,49520,2014-07-25,2014,0
54664,76561198081923658,True,570,2014-09-09,2014,0
46506,76561198031877091,True,252490,2014-01-12,2014,0
27536,CowOnWeed,True,220,2015-04-16,2015,0


In [12]:
userReviewsExplodedDf.to_parquet('../data/userReviewsExploded.parquet',index=False)